## Web Service Client

En este notebook vamos a consumir un servicio de machine learning. El servicio debe ser levantando con el script server.py por separado.

### EJERCICIO 1: 

Modifique el server.py para que acepte requests por GET. Modifique el codigo del cliente para enviar los datos por GET

In [17]:
# DEFINA SU CLIENTE GET AQUI:

# CODIGO PARA EL SERVICIO WEB:
"""
from flask import Flask
from flask_restful import Api, Resource, reqparse
import pickle
import numpy as np
import cv2, base64

# variables Flask
app = Flask(__name__)
api = Api(app)


# se carga el modelo de Logistic Regression del Notebook #3
pkl_filename = "ModeloLR.pkl"
with open(pkl_filename, 'rb') as file:
    #model = pickle.load(file)
    unpickler = pickle.Unpickler(file)
    model = unpickler.load()

class Predict(Resource):

    @staticmethod
    def get():
        #return 200
        
        parser = reqparse.RequestParser()
        parser.add_argument('image')
        # request para el modelo
        args = parser.parse_args() 
        im_bytes = base64.b64decode(args['image'])
        im_arr = np.frombuffer(im_bytes, dtype=np.uint8)  # im_arr is one-dim Numpy array
        img = cv2.imdecode(im_arr, flags=cv2.IMREAD_GRAYSCALE)
        img = img.reshape(-1)
        # prediccion
        out = {'Prediccion': int(model.predict(img.reshape(1, -1)))}
        return out,200


        
api.add_resource(Predict, '/predict')

if __name__ == '__main__':
    app.run(debug=True, port='1080')

"""

### EJERCICIO 2:

* Entrene un modelo de machine learning con MNIST y salve su modelo en un archivo pickle. [https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html]
* Modifique el server.py para que agregue la ruta /predict-number y que sea capaz de predecir si una imagen pertecene a un numero del 0 al 9
* Va a enviar la imagen desde el cliente (este notebook) como un base64 hacia el server.py
* el server.py va recibir la imagen la reconstruye en una imagen nuevamente y la manda al modelo de ML
* la imagen que ingresa al server.py debe salvarla en un folder (images)

In [96]:
import pickle
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

#print(__doc__)

# Author: Arthur Mensch <arthur.mensch@m4x.org>
# License: BSD 3 clause
# obtained from: https://scikit-learn.org/stable/auto_examples/linear_model/plot_sparse_logistic_regression_mnist.html

train_samples = 5000

# Load data from https://www.openml.org/d/554
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
#print(X[0].shape)

random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]
X = X.reshape((X.shape[0], -1))

X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=train_samples, test_size=10000)

#scaler = StandardScaler()
#X_train = scaler.fit_transform(X_train)

#print(X[0])

# Turn up tolerance for faster convergence
clf = LogisticRegression(
    C=50. / train_samples, penalty='l1', solver='saga', tol=0.1
)
clf.fit(X_train, y_train)


pkl_filename = "ModeloLR.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)
    
print('Model saved with name: ',pkl_filename) 


Model saved with name:  ModeloLR.pkl


In [150]:
import cv2, base64, requests
import matplotlib.pyplot as plt
img = cv2.imread('number5.png',0)
img= cv2.resize(img,(28,28))
#print(img.shape)
#plt.imshow(img,cmap='gray')
retval, buffer = cv2.imencode('.png',img)
img_to_txt = base64.b64encode(buffer)
#print(img_to_txt)

url = 'http://127.0.0.1:1080/predict'
body = {'image':img_to_txt}
response = requests.get(url, data=body)
print(response.json())
#print(response)



{'Prediccion': 2}
